In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "21018007" # 店舗ID
SPREADSHEET_KEY = '1fM-l5et0NogCACeM1zMGcrUMyFz9rHaJcmBqOcbPyfw' # スプレッドシートのID
pstart = 111 # パチンコの1台目
sstart = 2001 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p40row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p40table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "40":
                    p40table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "40":
                        p40table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.4RAW'!F" + str(int(p40row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p40table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025605', '111', '14359.0', '2408.0', '20', '5']
['08/19', '025865', '112', '20000.0', '3612.0', '61', '11']
['08/19', '025831', '113', '6154.0', '-1504.0', '5', '2']
['08/19', '025521', '115', '13590.0', '-1504.0', '11', '4']
['08/19', '025785', '116', '3846.0', '-3008.0', '0', '0']
['08/19', '025877', '117', '24359.0', '-902.0', '29', '8']
['08/19', '025875', '118', '13590.0', '-7522.0', '7', '7']
['08/19', '025521', '120', '8205.0', '904.0', '10', '3']
['08/19', '025844', '121', '24615.0', '25279.0', '45', '5']
['08/19', '025835', '122', '8974.0', '-902.0', '24', '4']
['08/19', '025891', '123', '13333.0', '5117.0', '28', '8']
['08/19', '025877', '125', '20256.0', '603.0', '28', '8']
['08/19', '025828', '126', '16923.0', '603.0', '30', '7']
['08/19', '025835', '127', '6154.0', '2709.0', '29', '4']
['08/19', '025765', '128', '10513.0', '-2106.0', '9', '5']
['08/19', '025841', '130', '15128.0', '-6920.0', '8', '5']
['08/19', '025865', '131', '18205.0', '-3610.0', '35', '10']


['08/19', '025694', '325', '15128.0', '-902.0', '17', '5']
['08/19', '025877', '326', '9744.0', '-3309.0', '7', '2']
['08/19', '025817', '327', '33846.0', '2107.0', '99', '13']
['08/19', '025844', '328', '5385.0', '904.0', '6', '1']
['08/19', '025835', '330', '25641.0', '3311.0', '96', '13']
['08/19', '025788', '331', '0', '0', '0', '0']
['08/19', '025779', '332', '8974.0', '3612.0', '14', '2']
['08/19', '025861', '333', '1026.0', '-1504.0', '0', '0']
['08/19', '025881', '335', '0', '0', '0', '0']
['08/19', '025876', '336', '13333.0', '-1805.0', '8', '2']
['08/19', '025667', '337', '1795.0', '-1504.0', '1', '1']
['08/19', '025707', '338', '5128.0', '-4212.0', '0', '0']
['08/19', '025779', '350', '2821.0', '-2406.0', '0', '0']
['08/19', '025625', '351', '1026.0', '-1504.0', '0', '0']
['08/19', '025668', '352', '2308.0', '603.0', '3', '1']
['08/19', '025645', '353', '28718.0', '5418.0', '19', '4']
['08/19', '025882', '355', '3590.0', '-2406.0', '2', '2']
['08/19', '025707', '356', '9744.

['08/19', '025824', '655', '0', '0', '0', '0']
['08/19', '025824', '656', '2564.0', '-2106.0', '0', '0']
['08/19', '025824', '657', '9231.0', '-6018.0', '1', '1']
['08/19', '025824', '658', '1795.0', '-1805.0', '0', '0']
['08/19', '025824', '660', '1026.0', '-1203.0', '0', '0']
['08/19', '025849', '661', '513.0', '-902.0', '0', '0']
['08/19', '025849', '662', '6923.0', '-5717.0', '0', '0']
['08/19', '025849', '663', '0', '0', '0', '0']
['08/19', '025849', '665', '11795.0', '-9629.0', '0', '0']
['08/19', '025870', '666', '769.0', '-1504.0', '0', '0']
['08/19', '025870', '667', '18718.0', '-14444.0', '1', '1']
['08/19', '025870', '668', '7692.0', '-4513.0', '1', '1']
['08/19', '025870', '670', '2051.0', '-2106.0', '0', '0']
['08/19', '025870', '671', '5128.0', '17154.0', '14', '2']
['08/19', '025764', '672', '2564.0', '-2106.0', '0', '0']
['08/19', '025764', '673', '4359.0', '-3309.0', '0', '0']
['08/19', '025764', '675', '5641.0', '1506.0', '4', '3']
['08/19', '025764', '676', '513.0', 

['08/19', '025890', '872', '5385.0', '9631.0', '15', '1']
['08/19', '025890', '873', '7692.0', '1205.0', '8', '1']
['08/19', '025890', '875', '9231.0', '4214.0', '13', '1']
['08/19', '025890', '876', '4103.0', '21367.0', '25', '1']
['08/19', '025890', '877', '6410.0', '-5717.0', '0', '0']
['08/19', '025890', '878', '0', '0', '0', '0']
['08/19', '025890', '880', '8462.0', '-6018.0', '2', '2']
['08/19', '025794', '881', '13590.0', '-5115.0', '4', '4']
['08/19', '025794', '882', '7179.0', '-1805.0', '3', '1']
['08/19', '025794', '883', '9231.0', '4515.0', '8', '3']
['08/19', '025852', '885', '1538.0', '-1203.0', '0', '0']
['08/19', '025852', '886', '1282.0', '-1203.0', '0', '0']
['08/19', '025852', '887', '6410.0', '6621.0', '23', '1']
['08/19', '025680', '888', '7692.0', '-4513.0', '1', '1']
['08/19', '025680', '1000', '3077.0', '-2406.0', '0', '0']
['08/19', '025680', '1001', '7692.0', '5418.0', '7', '1']
['08/19', '025430', '1002', '5128.0', '2107.0', '8', '3']
['08/19', '025430', '100

['08/19', '025814', '1187', '1026.0', '-1504.0', '0', '0']
['08/19', '025795', '1188', '769.0', '-1203.0', '0', '0']
['08/19', '025795', '1200', '3333.0', '-2707.0', '0', '0']
['08/19', '025550', '1201', '3590.0', '-3008.0', '0', '0']
['08/19', '025550', '1202', '0', '0', '0', '0']
['08/19', '025789', '1203', '0', '0', '0', '0']
['08/19', '025789', '1205', '0', '0', '0', '0']
['08/19', '025600', '1206', '0', '0', '0', '0']
['08/19', '025600', '1207', '0', '0', '0', '0']
['08/19', '025386', '1208', '0', '0', '0', '0']
['08/19', '025386', '1210', '5385.0', '-3911.0', '0', '0']
['08/19', '025493', '1211', '1026.0', '-1504.0', '0', '0']
['08/19', '025493', '1212', '0', '0', '0', '0']
['08/19', '025493', '1213', '0', '0', '0', '0']
['08/19', '025368', '1215', '3077.0', '-1504.0', '1', '1']
['08/19', '025368', '1216', '513.0', '-1203.0', '0', '0']
['08/19', '025696', '1217', '5641.0', '-4513.0', '0', '0']
['08/19', '025696', '1218', '9487.0', '-5416.0', '4', '1']
['08/19', '025544', '1220', 

...

['08/19', '120125', '2177', '292', '-370.0', '0', '0', '--']
['08/19', '120125', '2178', '2187', '278.0', '7', '8', '--']
['08/19', '120125', '2180', '520', '139.0', '2', '1', '--']
['08/19', '120125', '2181', '7714', '-278.0', '26', '21', '--']
['08/19', '120125', '2182', '2532', '-741.0', '7', '6', '--']
['08/19', '120125', '2183', '6708', '2037.0', '35', '11', '--']
['08/19', '120125', '2185', '969', '-417.0', '3', '0', '--']
['08/19', '120125', '2186', '2990', '-787.0', '10', '2', '--']
['08/19', '120125', '2187', '4460', '324.0', '19', '10', '--']
['08/19', '120125', '2188', '6158', '-880.0', '18', '15', '--']
['08/19', '120125', '2200', '658', '-556.0', '1', '0', '--']
['08/19', '120125', '2201', '1717', '185.0', '7', '4', '--']
['08/19', '120125', '2202', '5837', '370.0', '23', '13', '--']
['08/19', '120125', '2203', '4131', '787.0', '21', '6', '--']
['08/19', '120125', '2205', '990', '-370.0', '3', '0', '--']
['08/19', '120125', '2206', '969', '139.0', '3', '2', '--']
['08/19',

['08/19', '119996', '2380', '3699', '139.0', '14', '11', '--']
['08/19', '119996', '2381', '4253', '139.0', '16', '10', '--']
['08/19', '119996', '2382', '3528', '-1435.0', '8', '9', '--']
['08/19', '119996', '2383', '7098', '1296.0', '32', '19', '--']
['08/19', '119996', '2385', '3139', '-278.0', '12', '8', '--']
['08/19', '119996', '2386', '8707', '1667.0', '37', '29', '--']
['08/19', '119996', '2387', '2215', '-648.0', '6', '6', '--']
['08/19', '119996', '2388', '6719', '1343.0', '31', '21', '--']
['08/19', '119916', '2500', '4752', '1574.0', '22', '16', '--']
['08/19', '119916', '2501', '3630', '-185.0', '13', '11', '--']
['08/19', '119916', '2502', '3633', '-972.0', '10', '9', '--']
['08/19', '120033', '2503', '6674', '4259.0', '32', '12', '--']
['08/19', '120033', '2505', '6900', '1852.0', '28', '20', '--']
['08/19', '120033', '2506', '5582', '-3426.0', '11', '11', '--']
['08/19', '120033', '2507', '5632', '-1111.0', '15', '14', '--']
['08/19', '120033', '2508', '6176', '-1343.0'

['08/19', '120103', '2683', '2855', '-278.0', '28', '2', '10']
['08/19', '120103', '2685', '0', '0', '0', '0', '--']
['08/19', '120103', '2686', '739', '-694.0', '1', '2', '2']
['08/19', '120103', '2687', '3441', '3102.0', '48', '3', '13']
['08/19', '120103', '2688', '834', '-880.0', '1', '0', '3']
['08/19', '120103', '2700', '1010', '-1343.0', '0', '1', '4']
['08/19', '120103', '2701', '713', '-278.0', '3', '0', '2']
['08/19', '120103', '2702', '1561', '2130.0', '21', '1', '4']
['08/19', '120103', '2703', '3484', '3981.0', '41', '3', '15']
['08/19', '120103', '2705', '0', '0', '0', '0', '--']
['08/19', '120103', '2706', '1359', '231.0', '11', '1', '5']
['08/19', '120103', '2707', '3050', '370.0', '17', '1', '10']
['08/19', '120103', '2708', '2494', '2315.0', '43', '2', '9']
['08/19', '120104', '2710', '1384', '231.0', '0', '4', '9']
['08/19', '120104', '2711', '558', '-509.0', '0', '0', '1']
['08/19', '120104', '2712', '682', '-278.0', '0', '1', '3']
['08/19', '120104', '2713', '283',

['08/19', '120122', '3000', '0', '0', '0', '0', '--']
['08/19', '120122', '3001', '3213', '-556.0', '33', '5', '--']
['08/19', '120122', '3002', '4474', '7315.0', '99', '10', '--']
['08/19', '120122', '3003', '4185', '-1852.0', '34', '9', '--']
['08/19', '120122', '3005', '5274', '-1389.0', '45', '10', '--']
['08/19', '120122', '3006', '2774', '-1713.0', '19', '5', '--']
['08/19', '120122', '3007', '5521', '-2454.0', '40', '11', '--']
['08/19', '120122', '3008', '6030', '-1713.0', '50', '9', '--']
['08/19', '120122', '3010', '5691', '-1481.0', '48', '14', '--']
['08/19', '120122', '3011', '8069', '-2639.0', '64', '23', '--']
['08/19', '120122', '3012', '4366', '-370.0', '44', '13', '--']
['08/19', '120122', '3013', '4197', '-1296.0', '36', '11', '--']
['08/19', '120122', '3015', '4412', '-1991.0', '41', '10', '--']
['08/19', '120122', '3016', '5332', '278.0', '63', '10', '--']
['08/19', '120122', '3017', '4176', '-3148.0', '20', '9', '--']
['08/19', '120122', '3018', '2524', '-463.0', 

{'spreadsheetId': '1fM-l5et0NogCACeM1zMGcrUMyFz9rHaJcmBqOcbPyfw',
 'updates': {'spreadsheetId': '1fM-l5et0NogCACeM1zMGcrUMyFz9rHaJcmBqOcbPyfw',
  'updatedRange': 'S20RAW!F96077:M96604',
  'updatedRows': 528,
  'updatedColumns': 8,
  'updatedCells': 4224}}